<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Deep-Deterministic-Policy-Gradients-(DDPG)" data-toc-modified-id="Deep-Deterministic-Policy-Gradients-(DDPG)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Deep Deterministic Policy Gradients (DDPG)</a></span><ul class="toc-item"><li><span><a href="#Import-the-Necessary-Packages" data-toc-modified-id="Import-the-Necessary-Packages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import the Necessary Packages</a></span></li><li><span><a href="#Instantiate-the-Environment-and-Agent" data-toc-modified-id="Instantiate-the-Environment-and-Agent-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Instantiate the Environment and Agent</a></span><ul class="toc-item"><li><span><a href="#Examine-the-state-and-action-spaces" data-toc-modified-id="Examine-the-state-and-action-spaces-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Examine the state and action spaces</a></span></li></ul></li><li><span><a href="#Train-the-Agent-with-DDPG" data-toc-modified-id="Train-the-Agent-with-DDPG-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Train the Agent with DDPG</a></span><ul class="toc-item"><li><span><a href="#4.-Watch-a-Smart-Agent!" data-toc-modified-id="4.-Watch-a-Smart-Agent!-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>4. Watch a Smart Agent!</a></span></li><li><span><a href="#5.-Explore" data-toc-modified-id="5.-Explore-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>5. Explore</a></span></li></ul></li></ul></li></ul></div>

# Deep Deterministic Policy Gradients (DDPG)
---
In this notebook, we train DDPG with OpenAI Gym's BipedalWalker-v2 environment.

## Import the Necessary Packages

In [1]:
from unityagents import UnityEnvironment
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import importlib

import ddpg_agent
import prioritized_ddpg_agent

importlib.reload(prioritized_ddpg_agent)

<module 'prioritized_ddpg_agent' from 'C:\\Users\\thoma\\Documents\\GitHub\\deep-reinforcement-learning\\p2_continuous-control\\prioritized_ddpg_agent.py'>

## Instantiate the Environment and Agent

In [2]:
env = UnityEnvironment(file_name='Reacher_Windows_x86_64/Reacher.exe')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Creating functions for common environment processes

In [3]:
def environment_step(env, action, brain_name):
    env_info = env.step(action)[brain_name]
    next_states = env_info.vector_observations   # get the next state
    rewards = env_info.rewards                   # get the reward
    dones = env_info.local_done 
    
    return next_states, rewards, dones, None

def environment_reset(env, brain_name, train_mode=True):
    env_info = env.reset(train_mode=train_mode)[brain_name] # reset the environment
    
    return env_info

### Examine the state and action spaces

In [4]:
# reset the environment
env_info = environment_reset(env, brain_name)

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


## Train the Agent with DDPG

Run the code cell below to train the agent from scratch.  Alternatively, you can skip to the next code cell to load the pre-trained weights from file.

In [5]:
import prioritized_ddpg_agent
importlib.reload(prioritized_ddpg_agent)

#agent = prioritized_ddpg_agent.PrioritizedAgent(state_size=33, action_size=4, random_seed=42)
agent = ddpg_agent.Agent(state_size=33, action_size=4, random_seed=42)
agent.act(states, add_noise=False)

array([[-0.04892248, -0.06009788,  0.0061363 , -0.0487623 ]],
      dtype=float32)

In [6]:
import time

In [7]:
# DDPG function

def ddpg(n_episodes=3000, max_t=1000, solved_score=30.0, consec_episodes=100, print_every=1, train_mode=True,
         actor_path='actor_ckpt.pth', critic_path='critic_ckpt.pth'):
    """Deep Deterministic Policy Gradient (DDPG)
    
    Params
    ======
        n_episodes (int)      : maximum number of training episodes
        max_t (int)           : maximum number of timesteps per episode
        train_mode (bool)     : if 'True' set environment to training mode
        solved_score (float)  : min avg score over consecutive episodes
        consec_episodes (int) : number of consecutive episodes used to calculate score
        print_every (int)     : interval to display results
        actor_path (str)      : directory to store actor network weights
        critic_path (str)     : directory to store critic network weights

    """
    mean_scores = []                               # list of mean scores from each episode
    min_scores = []                                # list of lowest scores from each episode
    max_scores = []                                # list of highest scores from each episode
    best_score = -np.inf
    scores_window = deque(maxlen=consec_episodes)  # mean scores from most recent episodes
    moving_avgs = []                               # list of moving averages
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=train_mode)[brain_name] # reset environment
        states = env_info.vector_observations                   # get current state for each agent      
        scores = np.zeros(num_agents)                           # initialize score for each agent
        agent.reset()
        start_time = time.time()
        for t in range(max_t):
            actions = agent.act(states, add_noise=True)         # select an action
            env_info = env.step(actions)[brain_name]            # send actions to environment
            next_states = env_info.vector_observations          # get next state
            rewards = env_info.rewards                          # get reward
            dones = env_info.local_done                         # see if episode has finished
            # save experience to replay buffer, perform learning step at defined interval
            for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
                agent.step(state, action, reward, next_state, done, t)             
            states = next_states
            scores += rewards
                        
            if np.any(dones):                                   # exit loop when episode ends
                break

        duration = time.time() - start_time
        min_scores.append(np.min(scores))             # save lowest score for a single agent
        max_scores.append(np.max(scores))             # save highest score for a single agent        
        mean_scores.append(np.mean(scores))           # save mean score for the episode
        scores_window.append(mean_scores[-1])         # save mean score to window
        moving_avgs.append(np.mean(scores_window))    # save moving average
                
        if i_episode % print_every == 0:
            print('\rEpisode {} ({} sec)  -- \tMin: {:.1f}\tMax: {:.1f}\tMean: {:.1f}\tMov. Avg: {:.1f}'.format(\
                  i_episode, round(duration), min_scores[-1], max_scores[-1], mean_scores[-1], moving_avgs[-1]))
        
        if train_mode and mean_scores[-1] > best_score:
            torch.save(agent.actor_local.state_dict(), actor_path)
            torch.save(agent.critic_local.state_dict(), critic_path)
                  
        if moving_avgs[-1] >= solved_score and i_episode >= consec_episodes:
            print('\nEnvironment SOLVED in {} episodes!\tMoving Average ={:.1f} over last {} episodes'.format(\
                                    i_episode-consec_episodes, moving_avgs[-1], consec_episodes))            
            if train_mode:
                torch.save(agent.actor_local.state_dict(), actor_path)
                torch.save(agent.critic_local.state_dict(), critic_path)  
            break
            
    return mean_scores, moving_avgs

In [8]:
scores, avgs = ddpg()

Episode 1 (11 sec)  -- 	Min: 0.3	Max: 0.3	Mean: 0.3	Mov. Avg: 0.3
Episode 2 (13 sec)  -- 	Min: 0.2	Max: 0.2	Mean: 0.2	Mov. Avg: 0.3
Episode 3 (13 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Mov. Avg: 0.2
Episode 4 (13 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Mov. Avg: 0.1
Episode 5 (14 sec)  -- 	Min: 0.1	Max: 0.1	Mean: 0.1	Mov. Avg: 0.1
Episode 6 (14 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Mov. Avg: 0.1
Episode 7 (14 sec)  -- 	Min: 1.1	Max: 1.1	Mean: 1.1	Mov. Avg: 0.2
Episode 8 (14 sec)  -- 	Min: 0.1	Max: 0.1	Mean: 0.1	Mov. Avg: 0.2
Episode 9 (14 sec)  -- 	Min: 1.3	Max: 1.3	Mean: 1.3	Mov. Avg: 0.3
Episode 10 (14 sec)  -- 	Min: 0.2	Max: 0.2	Mean: 0.2	Mov. Avg: 0.3
Episode 11 (14 sec)  -- 	Min: 0.2	Max: 0.2	Mean: 0.2	Mov. Avg: 0.3
Episode 12 (14 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Mov. Avg: 0.3
Episode 13 (14 sec)  -- 	Min: 0.4	Max: 0.4	Mean: 0.4	Mov. Avg: 0.3
Episode 14 (14 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Mov. Avg: 0.3
Episode 15 (14 sec)  -- 	Min: 0.0	Max: 0.0	Mean: 0.0	Mov. Avg: 0.3
Epis

Episode 124 (22 sec)  -- 	Min: 6.2	Max: 6.2	Mean: 6.2	Mov. Avg: 2.8
Episode 125 (21 sec)  -- 	Min: 4.5	Max: 4.5	Mean: 4.5	Mov. Avg: 2.8
Episode 126 (22 sec)  -- 	Min: 7.5	Max: 7.5	Mean: 7.5	Mov. Avg: 2.9
Episode 127 (21 sec)  -- 	Min: 5.3	Max: 5.3	Mean: 5.3	Mov. Avg: 2.9
Episode 128 (21 sec)  -- 	Min: 5.6	Max: 5.6	Mean: 5.6	Mov. Avg: 3.0
Episode 129 (21 sec)  -- 	Min: 5.1	Max: 5.1	Mean: 5.1	Mov. Avg: 3.0
Episode 130 (21 sec)  -- 	Min: 3.3	Max: 3.3	Mean: 3.3	Mov. Avg: 3.0
Episode 131 (21 sec)  -- 	Min: 6.4	Max: 6.4	Mean: 6.4	Mov. Avg: 3.1
Episode 132 (21 sec)  -- 	Min: 3.4	Max: 3.4	Mean: 3.4	Mov. Avg: 3.1
Episode 133 (21 sec)  -- 	Min: 6.5	Max: 6.5	Mean: 6.5	Mov. Avg: 3.1
Episode 134 (21 sec)  -- 	Min: 6.9	Max: 6.9	Mean: 6.9	Mov. Avg: 3.2
Episode 135 (21 sec)  -- 	Min: 6.1	Max: 6.1	Mean: 6.1	Mov. Avg: 3.2
Episode 136 (21 sec)  -- 	Min: 9.1	Max: 9.1	Mean: 9.1	Mov. Avg: 3.3
Episode 137 (21 sec)  -- 	Min: 5.3	Max: 5.3	Mean: 5.3	Mov. Avg: 3.3
Episode 138 (21 sec)  -- 	Min: 8.5	Max: 8.5	Mean

Episode 242 (21 sec)  -- 	Min: 12.8	Max: 12.8	Mean: 12.8	Mov. Avg: 10.5
Episode 243 (21 sec)  -- 	Min: 13.8	Max: 13.8	Mean: 13.8	Mov. Avg: 10.5
Episode 244 (20 sec)  -- 	Min: 15.1	Max: 15.1	Mean: 15.1	Mov. Avg: 10.6
Episode 245 (21 sec)  -- 	Min: 16.2	Max: 16.2	Mean: 16.2	Mov. Avg: 10.8
Episode 246 (23 sec)  -- 	Min: 14.0	Max: 14.0	Mean: 14.0	Mov. Avg: 10.8
Episode 247 (21 sec)  -- 	Min: 18.8	Max: 18.8	Mean: 18.8	Mov. Avg: 11.0
Episode 248 (21 sec)  -- 	Min: 15.0	Max: 15.0	Mean: 15.0	Mov. Avg: 11.0
Episode 249 (23 sec)  -- 	Min: 15.7	Max: 15.7	Mean: 15.7	Mov. Avg: 11.1
Episode 250 (22 sec)  -- 	Min: 11.2	Max: 11.2	Mean: 11.2	Mov. Avg: 11.1
Episode 251 (22 sec)  -- 	Min: 13.9	Max: 13.9	Mean: 13.9	Mov. Avg: 11.1
Episode 252 (21 sec)  -- 	Min: 12.4	Max: 12.4	Mean: 12.4	Mov. Avg: 11.2
Episode 253 (21 sec)  -- 	Min: 8.5	Max: 8.5	Mean: 8.5	Mov. Avg: 11.2
Episode 254 (21 sec)  -- 	Min: 18.7	Max: 18.7	Mean: 18.7	Mov. Avg: 11.3
Episode 255 (21 sec)  -- 	Min: 11.8	Max: 11.8	Mean: 11.8	Mov. Avg: 

Episode 356 (22 sec)  -- 	Min: 32.9	Max: 32.9	Mean: 32.9	Mov. Avg: 19.9
Episode 357 (22 sec)  -- 	Min: 28.9	Max: 28.9	Mean: 28.9	Mov. Avg: 20.1
Episode 358 (22 sec)  -- 	Min: 25.2	Max: 25.2	Mean: 25.2	Mov. Avg: 20.2
Episode 359 (22 sec)  -- 	Min: 24.2	Max: 24.2	Mean: 24.2	Mov. Avg: 20.3
Episode 360 (22 sec)  -- 	Min: 19.3	Max: 19.3	Mean: 19.3	Mov. Avg: 20.3
Episode 361 (22 sec)  -- 	Min: 27.2	Max: 27.2	Mean: 27.2	Mov. Avg: 20.4
Episode 362 (22 sec)  -- 	Min: 25.4	Max: 25.4	Mean: 25.4	Mov. Avg: 20.5
Episode 363 (22 sec)  -- 	Min: 24.9	Max: 24.9	Mean: 24.9	Mov. Avg: 20.6
Episode 364 (22 sec)  -- 	Min: 30.2	Max: 30.2	Mean: 30.2	Mov. Avg: 20.8
Episode 365 (22 sec)  -- 	Min: 22.3	Max: 22.3	Mean: 22.3	Mov. Avg: 20.8
Episode 366 (22 sec)  -- 	Min: 25.3	Max: 25.3	Mean: 25.3	Mov. Avg: 20.9
Episode 367 (22 sec)  -- 	Min: 28.3	Max: 28.3	Mean: 28.3	Mov. Avg: 21.0
Episode 368 (22 sec)  -- 	Min: 34.0	Max: 34.0	Mean: 34.0	Mov. Avg: 21.2
Episode 369 (21 sec)  -- 	Min: 23.6	Max: 23.6	Mean: 23.6	Mov. Av

Episode 470 (23 sec)  -- 	Min: 31.6	Max: 31.6	Mean: 31.6	Mov. Avg: 29.7
Episode 471 (23 sec)  -- 	Min: 29.2	Max: 29.2	Mean: 29.2	Mov. Avg: 29.6
Episode 472 (23 sec)  -- 	Min: 31.4	Max: 31.4	Mean: 31.4	Mov. Avg: 29.7
Episode 473 (23 sec)  -- 	Min: 28.0	Max: 28.0	Mean: 28.0	Mov. Avg: 29.6
Episode 474 (23 sec)  -- 	Min: 24.5	Max: 24.5	Mean: 24.5	Mov. Avg: 29.6
Episode 475 (23 sec)  -- 	Min: 29.7	Max: 29.7	Mean: 29.7	Mov. Avg: 29.7
Episode 476 (23 sec)  -- 	Min: 31.0	Max: 31.0	Mean: 31.0	Mov. Avg: 29.7
Episode 477 (23 sec)  -- 	Min: 32.0	Max: 32.0	Mean: 32.0	Mov. Avg: 29.7
Episode 478 (24 sec)  -- 	Min: 33.4	Max: 33.4	Mean: 33.4	Mov. Avg: 29.8
Episode 479 (23 sec)  -- 	Min: 29.6	Max: 29.6	Mean: 29.6	Mov. Avg: 29.8
Episode 480 (23 sec)  -- 	Min: 31.2	Max: 31.2	Mean: 31.2	Mov. Avg: 29.9
Episode 481 (23 sec)  -- 	Min: 27.3	Max: 27.3	Mean: 27.3	Mov. Avg: 29.9
Episode 482 (22 sec)  -- 	Min: 27.0	Max: 27.0	Mean: 27.0	Mov. Avg: 29.9
Episode 483 (23 sec)  -- 	Min: 26.1	Max: 26.1	Mean: 26.1	Mov. Av

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [10]:
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

state = env.reset()
agent.reset()   
while True:
    action = agent.act(state)
    env.render()
    next_state, reward, done, _ = env.step(action)
    state = next_state
    if done:
        break
        
env.close()

RuntimeError: Error(s) in loading state_dict for Actor:
	Missing key(s) in state_dict: "batch_norm.weight", "batch_norm.bias", "batch_norm.running_mean", "batch_norm.running_var". 

In [7]:
env.close()

### 5. Explore

In this exercise, we have provided a sample DDPG agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster than this benchmark implementation.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task!
- Write your own DDPG implementation.  Use this code as reference only when needed -- try as much as you can to write your own algorithm from scratch.
- You may also like to implement prioritized experience replay, to see if it speeds learning.  
- The current implementation adds Ornsetein-Uhlenbeck noise to the action space.  However, it has [been shown](https://blog.openai.com/better-exploration-with-parameter-noise/) that adding noise to the parameters of the neural network policy can improve performance.  Make this change to the code, to verify it for yourself!
- Write a blog post explaining the intuition behind the DDPG algorithm and demonstrating how to use it to solve an RL environment of your choosing.  